In [3]:
import time
from jassair.utils import get_dataset_path, Datasets
import cv2 as cv
from IPython.display import display_png, Image, clear_output

## Prepare test images from captured images

In [4]:
target_x = 640
data_path = get_dataset_path(Datasets.REAL_LIFE_YOLO)

target_dir = data_path / "images"
target_dir.mkdir(exist_ok=True)

for i, image_path in enumerate(data_path.glob("*.jpg"), 1):
    image = cv.imread(image_path)
    
    h, w = image.shape[:2]
    h_2, w_2 = h // 2, w // 2
    x = min(h, w)
    x_2 = x // 2
    
    if x < target_x:
        raise ValueError(f"Image {image_path} is too small!, {image.shape[:2]}")
    
    h_start = h_2 - x_2
    h_stop = h_2 + x_2
    w_start = w_2 - x_2
    w_stop = w_2 + x_2        
    
    image = image[h_start:h_stop, w_start:w_stop]

    image = cv.resize(image, (target_x, target_x))
    cv.imwrite(f"{target_dir}/image_{i}.png", image) 

## Label the data

Data can easily be labeled. For every picture, you will be asked to label it. You will then return a string of the following format: `[SUIT][RANK]`

Example:
* `r0` = "Rose 10"
* `ek` = "Eichel Konig"
* `sa` = "Schelle Ass"
* `t6` = "Schilte 6"

The suits have the following labels:
* `e` = "Eichel"
* `r` = "Rose"
* `s` = "Schelle"
* `t` = "Schilte"

The ranks have the following labels:
* `6` = "6"
* `7` = "7"
* `8` = "8"
* `9` = "9"
* `0` = "10"
* `u` = "Under"
* `o` = "Ober"
* `k` = "König"
* `a` = "Ass"

In [48]:
SUIT = {
    "e": 0,
    "r": 9,
    "s": 18,
    "t": 27
}

RANK = {
    "0": 0,
    "6": 1,
    "7": 2,
    "8": 3,
    "9": 4,
    "a": 5,
    "k": 6,
    "o": 7,
    "u": 8,
}

def get_label(data: str) -> int:
    suit = data[0]
    rank = data[1]
    return SUIT.get(suit, -36) + RANK.get(rank, -36)

In [49]:
image_dir = target_dir
label_dir = data_path / "labels"
label_dir.mkdir(exist_ok=True)

redo = []

for image_path in image_dir.glob("*.png"):
    time.sleep(1)
    display_png(Image(image_path))
    filename = label_dir / f"{image_path.stem}.txt"
    time.sleep(3)
    a = get_label(input(f"What is this image?"))
    if a < 0:
        redo.append(image_path.name)
    else:
        with filename.open("w", encoding="utf-8") as f:
            f.write(str(a))
    clear_output()
if redo:
    print(f"Redo: {redo}")
else:
    print("Labeling finished")